Combining all the data tables together for analysis and predictive modelling

In [62]:
#import required packages
import os
import pathlib
import matplotlib.pyplot as plt
import numpy as np
import earthpy as et
import pandas as pd
import pickle
import random           #for test city selection

#read in the city info table
#os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','CITY_ADD_2022_09'))
#CITY_COUNTRY_lat_lon = pd.read_excel('CITY_COUNTRY_lat_lon.xlsx', index_col=None)

os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','MEAN_SUHI'))
CITY_COUNTRY_lat_lon = pd.read_excel('CITY_COUNTRY_lat_lon_mean.xlsx', index_col=None)

# read in the LULC percentage data 
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files'))
pickle_name = 'LULC_percentages_all_cities'
with open(pickle_name, 'rb') as f:
    LULC_percentages_all_cities = pickle.load(f)
    
# read in the roughness length data 
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','Roughness_Lengths'))
pickle_name = 'roughness_lengths_all_cities'
with open(pickle_name, 'rb') as f:
    roughness_lengths_all_cities = pickle.load(f)
    
# read    
    

In [42]:
CITY_COUNTRY_lat_lon

,CITY_COUNTRY,City,Lat,Lon,City_Area,City_Bound_xmin,City_Bound_xmax,City_Bound_ymin,City_Bound_ymax,koppen_climate,...,eccen_2018,major_ax_2019,minor_ax_2019,ell_theta_2019,eccen_2019,major_ax_2020,minor_ax_2020,ell_theta_2020,eccen_2020,std_dev_elevation
0,HUAMBO_ANGOLA,Huambo,-12.776110,15.739170,73,-6000,8000,-10000,6000,Cwb,...,0.358135,4747.419568,5089.130083,0.787862,0.360253,4822.026302,5261.610509,1.047506,0.400139,80.642464
1,ANAPOLIS_BRAZIL,Anapolis,-16.326670,-48.952780,92,-7000,10000,-10000,15000,Aw,...,0.816725,4312.304087,7850.274955,0.203487,0.835613,4312.304087,7850.274955,0.203487,0.835613,27.280540
2,FRANCA_BRAZIL,Franca,-20.538610,-47.400830,72,-8000,6000,-8000,8000,Cwa,...,0.608632,5507.885471,4440.010094,-0.701579,0.591754,5507.885471,4440.010094,-0.701579,0.591754,79.041565
3,PIRACICABA_BRAZIL,Piracicaba,-22.725280,-47.649170,75,-7000,8000,-7000,7000,Aw,...,0.827234,3721.993664,6624.451264,0.806659,0.827234,3721.993664,6624.451264,0.806659,0.827234,49.957558
4,RIBEIRAO_BRAZIL,Ribeirao,-21.177500,-47.810280,134,-8000,10000,-10000,11000,Aw,...,0.578572,5380.145805,6496.369960,-0.211900,0.560466,5380.145805,6496.369960,-0.211900,0.560466,29.645197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
107,FALOOJAH_IRAQ,Faloojah,33.355810,43.786120,29,-5000,4000,-5000,5000,BWh,...,0.581405,2877.878763,3537.154813,1.264488,0.581405,2877.878763,3537.154813,1.264488,0.581405,15.111382
108,SAMARRA_IRAQ,Samarra,34.196630,43.873900,19,-4000,6000,-5000,5000,BWh,...,0.752866,2245.877799,3312.148417,2.005438,0.734995,2245.877799,3312.148417,2.005438,0.734995,16.510735
109,POLOKWANE_SOUTH_AFRICA,Polokwane,-23.904480,29.468850,96,-13000,6000,-5000,13000,BWh,...,0.829796,7758.184645,4273.557325,-0.676743,0.834608,7800.428245,4459.303408,-0.708700,0.820481,95.319702
110,BATMAN_TURKEY,Batman,37.887380,41.132210,40,-7000,5500,-5000,8000,Csa,...,0.420518,4000.069135,4408.837594,0.433952,0.420518,4000.069135,4408.837594,0.433952,0.420518,138.839844


In [43]:
# remove cities with too small area
CITY_COUNTRY_lat_lon = CITY_COUNTRY_lat_lon[CITY_COUNTRY_lat_lon['Area_2002'] > 4]

## removing cities which have more than 60% of images unable to find a GSA fit
#remove_list = ['CUIABA_BRAZIL', 'MARINGA_BRAZIL', 'NDOLA_ZAMBIA',
#       'NAMPULA_MOZAMBIQUE', 'SOKOTO_NIGERIA', 'KANDAHAR_AFGHANISTAN',
#       'SHENMU_CHINA', 'ASYUT_EGYPT', 'ERODE_INDIA',
#       'NANDED_WAGHALA_INDIA', 'BAAQOOBAH_IRAQ', 'FALOOJAH_IRAQ',
#       'SAMARRA_IRAQ', 'POLOKWANE_SOUTH_AFRICA', 'BATMAN_TURKEY',
#       'GREENVILLE_USA','LAIBIN_CHINA']

#remember LAIBIN_CHINA
#CITY_COUNTRY_lat_lon = CITY_COUNTRY_lat_lon[~CITY_COUNTRY_lat_lon['CITY_COUNTRY'].isin(remove_list)].reset_index(drop = True)

In [44]:
# aqua only 
#create empty dataframe to append info into 
all_monthly_data_df = pd.DataFrame(data = None)

for i in range(len(CITY_COUNTRY_lat_lon)):
    CITY_COUNTRY = CITY_COUNTRY_lat_lon.CITY_COUNTRY[i]
    city_name = CITY_COUNTRY_lat_lon.City[i]
    #koppen_2_letter = CITY_COUNTRY_lat_lon.Koppen_Geiger_2_Letter[i]

    #read in the SUHI parameters  
    os.chdir(os.path.join('E:\\','City_Data','MODIS_8_day_LST', CITY_COUNTRY, 'MYD11A2'))    
    pickle_name = 'Parameters_Day_{}_{}.pkl'.format(CITY_COUNTRY, 'MYD11A2')
    with open(pickle_name, 'rb') as f:
        aqua_parameters_day = pickle.load(f)
    pickle_name = 'Parameters_Night_{}_{}.pkl'.format(CITY_COUNTRY, 'MYD11A2')
    with open(pickle_name, 'rb') as f:
        aqua_parameters_night = pickle.load(f)

    #read in urban and rural LSTs 
    #for analysis
#    pickle_name = 'urban_rural_lst_night_{}_{}.pkl'.format(CITY_COUNTRY, 'MYD11A2')
#    with open(pickle_name, 'rb') as f:
#        urban_rural_lst_night = pickle.load(f)
        
    #read in the climate parameters
    os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT', 'ERA5'))
    pickle_name = '{}_clim_df_overall_area'.format(CITY_COUNTRY)
    with open(pickle_name, 'rb') as f:
        clim_df = pickle.load(f)
    
    clim_df2 = clim_df.groupby(['year', 'month']).mean(numeric_only = True).reset_index(
                            )[['year', 'month', 'evap_fract', 'rh', 'tp','t2m','ssr','fal']]
    clim_df2 = clim_df2[(~clim_df2['year'].isin([2000,2001]))].reset_index(drop = True)
    
    
    # get the annual average ef
    
    annual_ef_df = clim_df2.groupby(['year'], as_index=False).mean(numeric_only = True)[['year','evap_fract']].rename(
        columns={"evap_fract": "annual_ef"})

    
    #################################### AQUA DAY #############################    
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #aqua_parameters_day
    aqua_parameters_day['year'] = pd.to_numeric(aqua_parameters_day['Year'], downcast='integer')
    aqua_parameters_day['month'] = pd.to_numeric(aqua_parameters_day['Month'], downcast='integer')
    
    group = (aqua_parameters_day.groupby(['year', 'month']).mean(numeric_only = True).reset_index()
                          [['year','month','day_a0','method_2_SUHI','quantile_75','footprint_area']])#'buffer_5km_SUHI',
    
    aqua_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    aqua_mon_averages = pd.merge(aqua_mon_averages, annual_ef_df, on =['year'], how = 'left')
    
    aqua_mon_averages['Overpass'] = '13:30'    
    aqua_mon_averages['City'] = city_name
    #aqua_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter
    
    #################################### AQUA NIGHT ##########################   
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #aqua_parameters_night
    aqua_parameters_night['year'] = pd.to_numeric(aqua_parameters_night['Year'], downcast='integer')
    aqua_parameters_night['month'] = pd.to_numeric(aqua_parameters_night['Month'], downcast='integer')
    
    group = (aqua_parameters_night.groupby(['year', 'month']).mean(numeric_only = True).reset_index()
                          [['year','month','night_a0','method_2_SUHI','quantile_75','footprint_area']])#,'buffer_5km_SUHI'
    
    aqua_night_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    aqua_night_mon_averages = pd.merge(aqua_night_mon_averages, annual_ef_df, on =['year'], how = 'left')
    
    aqua_night_mon_averages['Overpass'] = '01:30'    
    aqua_night_mon_averages['City'] = city_name
    #aqua_night_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter    

    #renames a0s to a0 so can be joined into same df
    aqua_mon_averages = aqua_mon_averages.rename(columns={"day_a0": "a0"})   
    aqua_night_mon_averages = aqua_night_mon_averages.rename(columns={"night_a0": "a0"}) 

    all_monthly_data_df = pd.concat([all_monthly_data_df, aqua_mon_averages])
    all_monthly_data_df = pd.concat([all_monthly_data_df, aqua_night_mon_averages])

##save the table as pickle
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities'))   
with open('aqua_no_evi_monthly_data_df.pkl', 'wb') as f:
    pickle.dump(all_monthly_data_df, f)
    

In [45]:
# add in the city area, rural roughness, eccentricity, LULC percentages
# create a df for the areas to merge

#add it LULC
LULC_to_add = LULC_percentages_all_cities[['CITY_COUNTRY','Year','Cropland, rainfed','Cropland, irrigated or post-flooding'
                                           , 'Bare areas','Water bodies', 'Grassland']]

LULC_to_add = LULC_to_add.rename(columns = {'Year':'year','Cropland, rainfed':'CROPLAND_RAIN',
                                 'Cropland, irrigated or post-flooding':'CROPLAND_IRR',
                                'Bare areas':'BARE','Water bodies':'WATER', 'Grassland':'GRASSLAND'})

LULC_to_add = pd.merge(LULC_to_add, CITY_COUNTRY_lat_lon[['CITY_COUNTRY','City']], on =['CITY_COUNTRY'], how = 'left')

LULC_to_add['year'] = pd.to_numeric(LULC_to_add['year'], downcast='integer')

all_monthly_data_df = pd.merge(all_monthly_data_df, LULC_to_add, on =['City','year'], how = 'left')


roughness_lengths_all_cities['year'] = pd.to_numeric(roughness_lengths_all_cities['Year'], downcast='integer')

all_monthly_data_df = pd.merge(all_monthly_data_df, roughness_lengths_all_cities, on =['CITY_COUNTRY','year'], how = 'left')


year_list = ['2000','2001','2002','2003','2004','2005','2006','2007','2008','2009','2010','2011','2012','2013',
                  '2014','2015','2016','2017','2018','2019','2020']
month_list = [1,2,3,4,5,6,7,8,9,10,11,12]

area_list = []
city_list = []
year_loop_list = []
eccentricity_list = []

for country_i, City in enumerate(CITY_COUNTRY_lat_lon.City):   
    for n, year in enumerate(year_list):
        area_list.append(CITY_COUNTRY_lat_lon['Area_{}'.format(year)][country_i])
        eccentricity_list.append(CITY_COUNTRY_lat_lon['eccen_{}'.format(year)][country_i])
        city_list.append(City)
        year_loop_list.append(year)
        
area_df = pd.DataFrame()
area_df['City'] = city_list
year_loop_list = list(map(int, year_loop_list))
area_df['year'] = year_loop_list
area_df['Area'] = area_list        
area_df['Eccentricity'] = eccentricity_list

# merge on
all_monthly_data_df = pd.merge(all_monthly_data_df, area_df, on =['City','year'], how = 'left')

##save the table as pickle
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities'))   
with open('aqua_no_evi_monthly_data_df.pkl', 'wb') as f:
    pickle.dump(all_monthly_data_df, f)

In [56]:
len(set(all_monthly_data_df2.City))

111

In [54]:
roughness_lengths_all_cities

,CITY_COUNTRY,Year,Roughness_Length_rural,Roughness_Length_all,year
0,HUAMBO_ANGOLA,2002,0.209333,0.213634,2002
1,HUAMBO_ANGOLA,2003,0.209442,0.21386,2003
2,HUAMBO_ANGOLA,2004,0.20925,0.213672,2004
3,HUAMBO_ANGOLA,2005,0.21045,0.215081,2005
4,HUAMBO_ANGOLA,2006,0.209491,0.214624,2006
...,...,...,...,...,...
14,GREENVILLE_USA,2016,0.220581,0.229093,2016
15,GREENVILLE_USA,2017,0.220671,0.22924,2017
16,GREENVILLE_USA,2018,0.216915,0.225664,2018
17,GREENVILLE_USA,2019,0.213378,0.222318,2019


In [47]:
all_monthly_data_df

,year,month,a0,method_2_SUHI,quantile_75,footprint_area,evap_fract,rh,tp,t2m,...,CROPLAND_RAIN,CROPLAND_IRR,BARE,WATER,GRASSLAND,Year,Roughness_Length_rural,Roughness_Length_all,Area,Eccentricity
0,2002,7,-2.428468,-1.461481,-1.182500,149.267847,0.451737,32.659958,0.000004,291.976410,...,15.789474,0.0,0.00000,0.000000,0.000000,2002,0.209333,0.213634,35,0.769305
1,2002,8,-2.160043,-1.388989,-1.200000,69.111796,0.396872,35.970322,0.000215,293.212769,...,15.789474,0.0,0.00000,0.000000,0.000000,2002,0.209333,0.213634,35,0.769305
2,2002,9,1.638151,-0.144017,1.250000,429.201804,0.416079,38.924397,0.000651,294.594177,...,15.789474,0.0,0.00000,0.000000,0.000000,2002,0.209333,0.213634,35,0.769305
3,2002,10,6.728467,1.661348,4.003333,219.252606,0.716956,75.767159,0.006287,292.602539,...,15.789474,0.0,0.00000,0.000000,0.000000,2002,0.209333,0.213634,35,0.769305
4,2002,11,3.847056,2.213450,4.130000,361.805099,0.730378,81.533249,0.007265,292.097290,...,15.789474,0.0,0.00000,0.000000,0.000000,2002,0.209333,0.213634,35,0.769305
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47692,2020,8,1.489493,1.149354,1.040000,1578.544148,0.831192,78.854408,0.005100,297.891876,...,0.764142,0.0,0.04562,0.901004,51.973084,2020,0.213655,0.222581,256,0.405440
47693,2020,9,1.621904,1.077187,1.285000,1606.640716,0.875226,74.295181,0.007240,294.176514,...,0.764142,0.0,0.04562,0.901004,51.973084,2020,0.213655,0.222581,256,0.405440
47694,2020,10,2.280195,0.279336,0.685000,366.037743,0.861734,75.925438,0.005163,290.422638,...,0.764142,0.0,0.04562,0.901004,51.973084,2020,0.213655,0.222581,256,0.405440
47695,2020,11,1.668680,0.543128,0.896667,616.665552,0.846493,72.002319,0.005137,285.825012,...,0.764142,0.0,0.04562,0.901004,51.973084,2020,0.213655,0.222581,256,0.405440


In [48]:
#interpolate the area df to give month values
month_list = [1,2,3,4,5,6,7,8,9,10,11,12]
area_df_monthly_interp = pd.DataFrame()

for count, year in enumerate(area_df.year):
    City = area_df.City[count]
    Area = area_df.Area[count]
    if year not in [2000,2020]:
        Area_year_before = area_df.Area[count - 1]
        Area_year_after = area_df.Area[count + 1]
        mon_diff_before = (Area - Area_year_before)/12
        mon_diff_after = (Area_year_after - Area)/12
        monthly_area_that_year = []
        for month_ind, month in enumerate(month_list):
            if month < 7:
                monthly_area_that_year.append(Area_year_before+(month+6)*mon_diff_before)
            else:
                monthly_area_that_year.append(Area+(month-6)*mon_diff_after)
     
    elif year == 2000:
        Area_year_after = area_df.Area[count + 1]
        mon_diff_after = (Area_year_after - Area)/12
        monthly_area_that_year = []
        for month_ind, month in enumerate(month_list):
            if month < 7:
                monthly_area_that_year.append(Area)
            else:
                monthly_area_that_year.append(Area+(month-6)*mon_diff_after)
    
    elif year == 2020:
        Area_year_before = area_df.Area[count - 1]
        mon_diff_before = (Area - Area_year_before)/12
        monthly_area_that_year = []
        for month_ind, month in enumerate(month_list):
            if month < 7:
                monthly_area_that_year.append(Area_year_before+(month+6)*mon_diff_before)
            else:
                monthly_area_that_year.append(Area)
    
    single_year = pd.DataFrame(data = month_list, columns = ['month'])
    single_year['City'] = City
    single_year['year'] = year
    single_year['Monthly_Area'] = monthly_area_that_year    
        
    area_df_monthly_interp = pd.concat([area_df_monthly_interp, single_year])
# merge on
all_monthly_data_df = pd.merge(all_monthly_data_df, area_df_monthly_interp, on =['City','year','month'], how = 'left')

##save the table as pickle
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities'))   
with open('aqua_no_evi_monthly_data_df.pkl', 'wb') as f:
    pickle.dump(all_monthly_data_df, f)

In [49]:
#remove 2021
all_monthly_data_df = all_monthly_data_df[all_monthly_data_df.year != 2021]

In [59]:
all_evi_df

,Aqua_Filename,Month,Year,mean_NDVI_x,mean_EVI_x,usable_pixel_%_x,urban_usable_pixel_%_x,Aqua_qc_pass,qc_pass_all_pix_x,qc_pass_urb_pix_x,...,urb_mean_evi,pixel_reliability_percent,City,mean_NDVI,mean_EVI,usable_pixel_%,urban_usable_pixel_%,qc_pass_overall,qc_pass_all_pix,qc_pass_urb_pix
0,MYD13A3.A2002182.h19v10.006.2015154143627.hdf,7,2002,0.303408,0.167875,5.701754,25.000000,1.0,1.0,1.0,...,0.175464,99.498747,Huambo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MYD13A3.A2002213.h19v10.006.2015150083433.hdf,8,2002,0.304745,0.156461,20.802005,27.777778,1.0,1.0,1.0,...,NaN,NaN,Huambo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MYD13A3.A2002244.h19v10.006.2015151113359.hdf,9,2002,0.316847,0.174465,3.070175,22.222222,1.0,1.0,1.0,...,NaN,NaN,Huambo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MYD13A3.A2002274.h19v10.006.2015152043021.hdf,10,2002,0.375957,0.196343,2.318296,0.000000,1.0,1.0,1.0,...,NaN,NaN,Huambo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MYD13A3.A2002305.h19v10.006.2015153092025.hdf,11,2002,0.472847,0.258728,3.571429,0.000000,1.0,1.0,1.0,...,NaN,NaN,Huambo,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
217,MYD13A3.A2020214.h11v05.006.2020260222402.hdf,8,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.424437,80.348996,Greenville,0.786203,0.525640,16.571624,9.132420,1.0,1.0,1.0
218,MYD13A3.A2020245.h11v05.006.2020289195137.hdf,9,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.362973,97.285584,Greenville,0.772717,0.443540,96.270529,77.397260,0.0,0.0,0.0
219,MYD13A3.A2020275.h11v05.006.2020315002328.hdf,10,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.322621,97.057482,Greenville,0.720803,0.396866,78.535584,44.977169,0.0,0.0,1.0
220,MYD13A3.A2020306.h11v05.006.2020347040143.hdf,11,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.283612,98.597172,Greenville,0.618864,0.315865,89.290602,66.894977,0.0,0.0,0.0


In [57]:
# add in the evi data and elevation
# create a dataframe with all the EVI data
all_evi_df = pd.DataFrame(data = None)

for i in range(len(CITY_COUNTRY_lat_lon)):
    CITY_COUNTRY = CITY_COUNTRY_lat_lon['CITY_COUNTRY'][i] 
    city_name = CITY_COUNTRY_lat_lon['City'][i]  
    #open the city EVI df
    os.chdir(os.path.join('E:\\','City_Data','MODIS_NDVI', CITY_COUNTRY))
    pickle_name = 'vi_means_df_{}.pkl'.format(CITY_COUNTRY)
    with open(pickle_name, 'rb') as f:
        vi_means_df = pickle.load(f)
    #add the city name
    vi_means_df['City'] = city_name
    #append to overall df
    all_evi_df = pd.concat([all_evi_df, vi_means_df])

#change year/ month dtypes to int so can be matched on merge
all_evi_df['Year'] = all_evi_df['Year'].apply(int)
all_evi_df['Month'] = all_evi_df['Month'].apply(int)
all_monthly_data_df['Year'] = all_monthly_data_df['year'].apply(int)
all_monthly_data_df['Month'] = all_monthly_data_df['month'].apply(int)
    
#merge the datasets
all_monthly_data_df2 = pd.merge(all_monthly_data_df, all_evi_df, how='left', left_on=['City','Year','Month'], 
                                right_on = ['City','Year','Month'])    
      
#drop useless columns 
all_monthly_data_df2 = all_monthly_data_df2.drop(columns=['Year', 'Month','Aqua_Filename', 'Terra_Filename'])

#drop where the pixel reliability is less than 30%
#all_monthly_data_df2 = all_monthly_data_df2[all_monthly_data_df2.pixel_reliability_percent >= 0.7]
all_monthly_data_df2 = all_monthly_data_df2[all_monthly_data_df2.qc_pass_all_pix == 0]
#qc_pass_all_pix

# add in elevation data
all_monthly_data_df2 = pd.merge(all_monthly_data_df2, CITY_COUNTRY_lat_lon[['CITY_COUNTRY','mean_elevation_rural',
                        'mean_elevation_city', 'std_elevation_rural','std_elevation_city']], how='left', 
                                left_on=['CITY_COUNTRY'], right_on = ['CITY_COUNTRY'])    


#save the new dataset
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities')) 
with open('aqua_all_monthly_data_df2.pkl', 'wb') as f:
    pickle.dump(all_monthly_data_df2, f)

In [61]:
all_monthly_data_df

,year,month,a0,method_2_SUHI,quantile_75,footprint_area,evap_fract,rh,tp,t2m,...,BARE,WATER,GRASSLAND,Year,Roughness_Length_rural,Roughness_Length_all,Area,Eccentricity,Monthly_Area,Month
0,2002,7,-2.428468,-1.461481,-1.182500,149.267847,0.451737,32.659958,0.000004,291.976410,...,0.00000,0.000000,0.000000,2002,0.209333,0.213634,35,0.769305,35.083333,7
1,2002,8,-2.160043,-1.388989,-1.200000,69.111796,0.396872,35.970322,0.000215,293.212769,...,0.00000,0.000000,0.000000,2002,0.209333,0.213634,35,0.769305,35.166667,8
2,2002,9,1.638151,-0.144017,1.250000,429.201804,0.416079,38.924397,0.000651,294.594177,...,0.00000,0.000000,0.000000,2002,0.209333,0.213634,35,0.769305,35.250000,9
3,2002,10,6.728467,1.661348,4.003333,219.252606,0.716956,75.767159,0.006287,292.602539,...,0.00000,0.000000,0.000000,2002,0.209333,0.213634,35,0.769305,35.333333,10
4,2002,11,3.847056,2.213450,4.130000,361.805099,0.730378,81.533249,0.007265,292.097290,...,0.00000,0.000000,0.000000,2002,0.209333,0.213634,35,0.769305,35.416667,11
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47692,2020,8,1.489493,1.149354,1.040000,1578.544148,0.831192,78.854408,0.005100,297.891876,...,0.04562,0.901004,51.973084,2020,0.213655,0.222581,256,0.405440,256.000000,8
47693,2020,9,1.621904,1.077187,1.285000,1606.640716,0.875226,74.295181,0.007240,294.176514,...,0.04562,0.901004,51.973084,2020,0.213655,0.222581,256,0.405440,256.000000,9
47694,2020,10,2.280195,0.279336,0.685000,366.037743,0.861734,75.925438,0.005163,290.422638,...,0.04562,0.901004,51.973084,2020,0.213655,0.222581,256,0.405440,256.000000,10
47695,2020,11,1.668680,0.543128,0.896667,616.665552,0.846493,72.002319,0.005137,285.825012,...,0.04562,0.901004,51.973084,2020,0.213655,0.222581,256,0.405440,256.000000,11


In [51]:
# rename the columns
#all_monthly_data_df2
all_monthly_data_df2 = all_monthly_data_df2.rename(columns={"a0": "SUHI_PEAK_GSA", "method_2_SUHI": "SUHI_MEAN", "quantile_75": "SUHI_PEAK_QUANTILE"
             , "footprint_area": "SUHI_FP", "evap_fract": "EF", "rh": "RH", "tp": "TP", "t2m": "T2M", "ssr": "SSR",
               'Eccentricity':'ECC', 'rur_mean_evi': 'EVI_R','urb_mean_evi': 'EVI_U', 
                'mean_elevation_rural':'ELEVATION_R', 'mean_elevation_city':'ELEVATION_U',
                 'std_elevation_rural':'STD_ELEVATION_R','std_elevation_city':'STD_ELEVATION_U',
                'Roughness_Length_rural':'ROUGHNESS_LENGTH_R'})

all_monthly_data_df2 = all_monthly_data_df2[(all_monthly_data_df2.SUHI_PEAK_GSA > - 20)].reset_index(drop = True)
all_monthly_data_df2 = all_monthly_data_df2[(all_monthly_data_df2.SUHI_PEAK_GSA < 20)].reset_index(drop = True)
#save the new dataset
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities')) 
with open('aqua_all_monthly_data_df2.pkl', 'wb') as f:
    pickle.dump(all_monthly_data_df2, f)

In [58]:
set(all_monthly_data_df2.City)

{'Akola',
 'Al Obeid (Al Ubayyid)',
 'Al-Raqqa',
 'Alwar',
 'Anapolis',
 'Anshun',
 'Asyut',
 'Baaqoobah',
 'Bahawalpur',
 'Bathinda',
 'Batman',
 'Bijapur',
 'Bikaner',
 'Birmingham',
 'Birnin Kebbi',
 'Bloemfontein',
 'Bobo_Dioulasso',
 'Bozhou',
 'Bulawayo',
 'Buraydah',
 'Campo_Grande',
 'Caruaru',
 'Cascavel',
 'Cuiaba',
 'Cuito',
 'Dayton',
 'Dazhou',
 'Dengzhou',
 'Deyang',
 'Diwaniyah',
 'El Djelfa',
 'Erode',
 'Faloojah',
 'Fayetteville',
 'Franca',
 'Fuyang',
 'Gaoan',
 'Gaya',
 'Gombe',
 'Greenville',
 "Ha'il",
 'Hafar_al_Batin',
 'Hechi',
 'Heze',
 'Hisar',
 'Huaihua',
 'Jalandhar',
 'Kandahar',
 'Kassala',
 'Kasur',
 'Khammam',
 'Kikwit',
 'Kirkuk',
 'Lampang',
 'Lexington-Fayette',
 'Likasi',
 'Linqing',
 'Liuyang',
 'Londrina',
 'Luohe',
 'Malanje',
 'Maringa',
 'Marrakech',
 'Mathura',
 'Montes Claros',
 'Montgomery',
 'Muzaffarnagar',
 'Mymensingh',
 'Nakhon Ratchasima',
 'Nampula',
 'Nanded Waghala',
 'Ndola',
 'Ningguo',
 'Nyala',
 'Ogbomosho',
 'Parakou',
 'Parbhani

In [130]:
# Deciding which cities should be in the test dataset

#cities with the lowest a0 and highest a0 should be in the training
max_SUHI_PEAK_GSA_city = all_monthly_data_df2[all_monthly_data_df2.SUHI_PEAK_GSA == 
                                              all_monthly_data_df2.SUHI_PEAK_GSA.max()].City.values[0]

max_SUHI_MEAN_city = all_monthly_data_df2[all_monthly_data_df2.SUHI_MEAN == 
                                          all_monthly_data_df2.SUHI_MEAN.max()].City.values[0]

min_SUHI_PEAK_GSA_city = all_monthly_data_df2[all_monthly_data_df2.SUHI_PEAK_GSA == 
                                   all_monthly_data_df2.SUHI_PEAK_GSA.min()].City.values[0]

min_SUHI_MEAN_city = all_monthly_data_df2[all_monthly_data_df2.SUHI_MEAN == 
                                          all_monthly_data_df2.SUHI_MEAN.min()].City.values[0]

in_training = [max_SUHI_PEAK_GSA_city, max_SUHI_MEAN_city, min_SUHI_PEAK_GSA_city, min_SUHI_MEAN_city]

#create a list of the candiates
test_candidates = CITY_COUNTRY_lat_lon[~CITY_COUNTRY_lat_lon.City.isin(in_training)].reset_index()

#get one letter koppen geiger
koppen_1_list = []
for i in range(len(test_candidates)):
    koppen_1_letter = test_candidates.koppen_climate[i][0]
    koppen_1_list.append(koppen_1_letter)
    
test_candidates['koppen_1_letter'] = koppen_1_list

#randomly pick test cities from each group
#9 cities to be chosen
#5 from A, 7 from B, 6 from C
num_A = len(test_candidates[test_candidates['koppen_1_letter'] == 'A'])
num_B = len(test_candidates[test_candidates['koppen_1_letter'] == 'B'])
num_C = len(test_candidates[test_candidates['koppen_1_letter'] == 'C'])
num_D = len(test_candidates[test_candidates['koppen_1_letter'] == 'D'])

# generate random test cities
test_cities = random.sample([*set(test_candidates[test_candidates['koppen_1_letter'] == 'A'].City.values)], k=5)
test_cities = test_cities + (random.sample([*set(test_candidates[test_candidates['koppen_1_letter'] == 'B'].City.values)],
                                           k=5))
test_cities = test_cities + (random.sample([*set(test_candidates[test_candidates['koppen_1_letter'] == 'C'].City.values)],
                                           k=6))

In [132]:
test_candidates

,index,CITY_COUNTRY,City,Lat,Lon,City_Area,City_Bound_xmin,City_Bound_xmax,City_Bound_ymin,City_Bound_ymax,...,major_ax_2019,minor_ax_2019,ell_theta_2019,eccen_2019,major_ax_2020,minor_ax_2020,ell_theta_2020,eccen_2020,std_dev_elevation,koppen_1_letter
0,0,HUAMBO_ANGOLA,Huambo,-12.776110,15.739170,73,-6000,8000,-10000,6000,...,4747.419568,5089.130083,0.787862,0.360253,4822.026302,5261.610509,1.047506,0.400139,80.642464,C
1,1,ANAPOLIS_BRAZIL,Anapolis,-16.326670,-48.952780,92,-7000,10000,-10000,15000,...,4312.304087,7850.274955,0.203487,0.835613,4312.304087,7850.274955,0.203487,0.835613,27.280540,A
2,2,FRANCA_BRAZIL,Franca,-20.538610,-47.400830,72,-8000,6000,-8000,8000,...,5507.885471,4440.010094,-0.701579,0.591754,5507.885471,4440.010094,-0.701579,0.591754,79.041565,C
3,3,PIRACICABA_BRAZIL,Piracicaba,-22.725280,-47.649170,75,-7000,8000,-7000,7000,...,3721.993664,6624.451264,0.806659,0.827234,3721.993664,6624.451264,0.806659,0.827234,49.957558,A
4,4,RIBEIRAO_BRAZIL,Ribeirao,-21.177500,-47.810280,134,-8000,10000,-10000,11000,...,5380.145805,6496.369960,-0.211900,0.560466,5380.145805,6496.369960,-0.211900,0.560466,29.645197,A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88,91,LEXINGTON_FAYETTE_USA,Lexington-Fayette,38.029632,-84.494642,178,-9500,10000,-10000,10000,...,6392.512827,6849.572225,1.421412,0.359171,6392.512827,6849.572225,1.421412,0.359171,22.746433,C
89,92,MONTGOMERY_USA,Montgomery,32.372034,-86.304457,208,-7500,16500,-10000,7000,...,5974.609929,9129.936085,1.543021,0.756150,5974.609929,9129.936085,1.543021,0.756150,30.125141,C
90,93,VISALIA_USA,Visalia,36.324100,-119.307347,90,-11000,7500,-5000,8000,...,3694.839410,6664.437276,1.412804,0.832243,3694.839410,6664.437276,1.412804,0.832243,73.151581,B
91,94,WICHITA_USA,Wichita,37.688848,-97.336226,336,-15000,15000,-15500,14000,...,9298.568243,9790.571909,1.132905,0.313018,9298.568243,9790.571909,1.132905,0.313018,16.668718,C


In [21]:
test_cities
#['Sikasso', 'Anapolis', 'Campo_Grande', 'Bobo_Dioulasso', 'Diwaniyah', 'Al_Obeid', 'Akola', 'Potiskum','Cascavel']

['Uberaba',
 'Rio Branco',
 'Sikasso',
 'Gombe',
 'Anapolis',
 'Kassala',
 'Yulin, Shaanxi',
 'Bikaner',
 'Zhuozhou',
 'Zinder',
 'Huaihua',
 'Birmingham',
 'Dayton',
 'Shangqiu',
 'Lexington-Fayette',
 'Gaya']

In [19]:
num_A ,num_B ,num_C ,num_D

(24, 35, 32, 1)

In [7]:
#add in later? 'buffer_5km_SUHI','quantile_a0','footprint_area'

#create empty dataframe to append info into 
all_monthly_data_df = pd.DataFrame(data = None)

for i in range(len(CITY_COUNTRY_lat_lon)):
    CITY_COUNTRY = CITY_COUNTRY_lat_lon.CITY_COUNTRY[i]
    city_name = CITY_COUNTRY_lat_lon.City[i]
    koppen_2_letter = CITY_COUNTRY_lat_lon.Koppen_Geiger_2_Letter[i]

    #read in the SUHI parameters
    os.chdir(os.path.join('D:\\','MODIS_8_day_LST', CITY_COUNTRY, 'MOD11A2'))
    pickle_name = 'Parameters_Day_{}_{}.pkl'.format(CITY_COUNTRY, 'MOD11A2')
    with open(pickle_name, 'rb') as f:
        terra_parameters_day = pickle.load(f)
    pickle_name = 'Parameters_Night_{}_{}.pkl'.format(CITY_COUNTRY, 'MOD11A2')
    with open(pickle_name, 'rb') as f:
        terra_parameters_night = pickle.load(f)   
    os.chdir(os.path.join('D:\\','MODIS_8_day_LST', CITY_COUNTRY, 'MYD11A2'))    
    pickle_name = 'Parameters_Day_{}_{}.pkl'.format(CITY_COUNTRY, 'MYD11A2')
    with open(pickle_name, 'rb') as f:
        aqua_parameters_day = pickle.load(f)
    pickle_name = 'Parameters_Night_{}_{}.pkl'.format(CITY_COUNTRY, 'MYD11A2')
    with open(pickle_name, 'rb') as f:
        aqua_parameters_night = pickle.load(f)

    #read in the climate parameters
    os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT', 'ERA5', CITY_COUNTRY))
    pickle_name = '{}_clim_rur_urb'.format(city_name)
    with open(pickle_name, 'rb') as f:
        clim_df = pickle.load(f)
    
    clim_df2 = clim_df.reset_index()[['year', 'month', 'overall_evap_fract', 'overall_rh', 
                                      'urban_evap_fract', 'urban_rh', 'rural_evap_fract', 'rural_rh',
                                     'overall_tp','rural_tp','urban_tp','overall_t2m','rural_t2m','urban_t2m',
                                     'overall_ssr','rural_ssr','urban_ssr']]
    
    #################################### TERRA DAY ##############################    
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #convert the year and month columns to integer so they are equal for the merge
    terra_parameters_day['year'] = pd.to_numeric(terra_parameters_day['Year'], downcast='integer')
    terra_parameters_day['month'] = pd.to_numeric(terra_parameters_day['Month'], downcast='integer')
    
    group = (terra_parameters_day.groupby(['year', 'month']).mean().reset_index()
                          [['year','month','day_a0','method_2_SUHI','quantile_a0']])
    
    terra_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    
    terra_mon_averages['Overpass'] = '10:30'    
    terra_mon_averages['City'] = city_name
    terra_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter
    
    #################################### AQUA DAY #############################    
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #aqua_parameters_day
    aqua_parameters_day['year'] = pd.to_numeric(aqua_parameters_day['Year'], downcast='integer')
    aqua_parameters_day['month'] = pd.to_numeric(aqua_parameters_day['Month'], downcast='integer')
    
    group = (aqua_parameters_day.groupby(['year', 'month']).mean().reset_index()
                          [['year','month','day_a0','method_2_SUHI','quantile_a0']])
    
    aqua_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    
    aqua_mon_averages['Overpass'] = '13:30'    
    aqua_mon_averages['City'] = city_name
    aqua_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter
    
    ################################### TERRA NIGHT ###########################    
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #terra_parameters_night
    terra_parameters_night['year'] = pd.to_numeric(terra_parameters_night['Year'], downcast='integer')
    terra_parameters_night['month'] = pd.to_numeric(terra_parameters_night['Month'], downcast='integer')
    
    group = (terra_parameters_night.groupby(['year', 'month']).mean().reset_index()
                          [['year','month','night_a0','method_2_SUHI','quantile_a0']])
    
    terra_night_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    
    terra_night_mon_averages['Overpass'] = '22:30'    
    terra_night_mon_averages['City'] = city_name
    terra_night_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter
    
    #################################### AQUA NIGHT ##########################   
    #add in the grouping the the parameters dataset to get a0 and mean SUHI
    #aqua_parameters_night
    aqua_parameters_night['year'] = pd.to_numeric(aqua_parameters_night['Year'], downcast='integer')
    aqua_parameters_night['month'] = pd.to_numeric(aqua_parameters_night['Month'], downcast='integer')
    
    group = (aqua_parameters_night.groupby(['year', 'month']).mean().reset_index()
                          [['year','month','night_a0','method_2_SUHI','quantile_a0']])
    
    aqua_night_mon_averages = pd.merge(group, clim_df2, on =['year', 'month'], how = 'inner').drop_duplicates()
    
    aqua_night_mon_averages['Overpass'] = '01:30'    
    aqua_night_mon_averages['City'] = city_name
    aqua_night_mon_averages['Koppen_Geiger_2_Letter'] = koppen_2_letter    

    #renames a0s to a0 so can be joined into same df
    terra_mon_averages = terra_mon_averages.rename(columns={"day_a0": "a0"})
    aqua_mon_averages = aqua_mon_averages.rename(columns={"day_a0": "a0"})
    terra_night_mon_averages = terra_night_mon_averages.rename(columns={"night_a0": "a0"})
    aqua_night_mon_averages = aqua_night_mon_averages.rename(columns={"night_a0": "a0"}) 

    all_monthly_data_df = all_monthly_data_df.append(terra_mon_averages)
    all_monthly_data_df = all_monthly_data_df.append(aqua_mon_averages)
    all_monthly_data_df = all_monthly_data_df.append(terra_night_mon_averages)
    all_monthly_data_df = all_monthly_data_df.append(aqua_night_mon_averages)

#save the table as pickle
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities'))   
with open('all_monthly_data_df.pkl', 'wb') as f:
    pickle.dump(all_monthly_data_df, f)

In [10]:
#add in the evi data
#create a dataframe with all the EVI data
all_evi_df = pd.DataFrame(data = None)

for i in range(len(CITY_COUNTRY_lat_lon)):
    CITY_COUNTRY = CITY_COUNTRY_lat_lon['CITY_COUNTRY'][i] 
    city_name = CITY_COUNTRY_lat_lon['City'][i]  
    #open the city EVI df
    os.chdir(os.path.join('D:\\','MODIS_NDVI', CITY_COUNTRY))
    pickle_name = 'vi_means_df_{}.pkl'.format(CITY_COUNTRY)
    with open(pickle_name, 'rb') as f:
        vi_means_df = pickle.load(f)
    #add the city name
    vi_means_df['City'] = city_name
    #append to overall df
    all_evi_df = all_evi_df.append(vi_means_df)

#change year/ month dtypes to int so can be matched on merge
all_evi_df['Year'] = all_evi_df['Year'].apply(int)
all_evi_df['Month'] = all_evi_df['Month'].apply(int)
all_monthly_data_df['Year'] = all_monthly_data_df['year'].apply(int)
all_monthly_data_df['Month'] = all_monthly_data_df['month'].apply(int)
    
#merge the datasets
all_monthly_data_df2 = pd.merge(all_monthly_data_df, all_evi_df, how='left', left_on=['City','Year','Month'], 
                                right_on = ['City','Year','Month'])    
      
#drop useless columns 
all_monthly_data_df2 = all_monthly_data_df2.drop(columns=['Year', 'Month','Aqua_Filename', 'Terra_Filename',
                                                         'Terra_Filename_top','Aqua_Filename_top'])

#save the new dataset
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities')) 
with open('all_monthly_data_df2.pkl', 'wb') as f:
    pickle.dump(all_monthly_data_df2, f)

In [ ]:
## adding the extention latlon table on

In [28]:
#read in the city info table
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','CITY_ADD_2022_09'))
CITY_COUNTRY_lat_lon = pd.read_excel('CITY_COUNTRY_lat_lon.xlsx', index_col=None)

#read in the extention table 
#os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','CITY_ADD_2022_06'))
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','CITY_ADD_2022_09'))
CITY_COUNTRY_lat_lon_EXT = pd.read_excel('CITY_COUNTRY_lat_lon_EXT.xlsx', index_col=None)
all_cities = pd.read_excel('all_cities.xlsx', index_col=None)
#new_cities_grid_refs = pd.read_excel('grid_refs.xlsx', index_col=None)
all_cities_std_ele = all_cities[['Latitude', 'Longitude','std_dev_elevation']]
CITY_COUNTRY_lat_lon_EXT = CITY_COUNTRY_lat_lon_EXT.drop(columns=['Unnamed: 0'])

In [29]:
CITY_COUNTRY_lat_lon = pd.concat([CITY_COUNTRY_lat_lon, CITY_COUNTRY_lat_lon_EXT])

In [30]:
CITY_COUNTRY_lat_lon = pd.merge(CITY_COUNTRY_lat_lon, all_cities_std_ele,  how='left', 
                                left_on=['Lat','Lon'], right_on = ['Latitude','Longitude'])
#duplicates due to city name
remove_city_list = (['Anápolis','Cuiabá','Ribeirão Preto','Campo Grande','Bobo-Dioulasso','Hafar al-Batin',
                        'Maringá','São José do Rio Preto'])
CITY_COUNTRY_lat_lon = CITY_COUNTRY_lat_lon[~(CITY_COUNTRY_lat_lon['City']).isin(remove_city_list)] 
CITY_COUNTRY_lat_lon= CITY_COUNTRY_lat_lon[CITY_COUNTRY_lat_lon['CITY_COUNTRY']!= 'RIBEIRAO_PRETO_BRAZIL']

In [62]:
#duplicates due to city name
remove_city_list = (['Anápolis','Cuiabá','Ribeirão Preto','Campo Grande','Bobo-Dioulasso','Hafar al-Batin',
                        'Maringá','São José do Rio Preto'])
CITY_COUNTRY_lat_lon = CITY_COUNTRY_lat_lon[~(CITY_COUNTRY_lat_lon['City']).isin(remove_city_list)] 
CITY_COUNTRY_lat_lon= CITY_COUNTRY_lat_lon[CITY_COUNTRY_lat_lon['CITY_COUNTRY']!= 'RIBEIRAO_PRETO_BRAZIL']
CITY_COUNTRY_lat_lon = CITY_COUNTRY_lat_lon[~np.isnan(CITY_COUNTRY_lat_lon['std_dev_elevation'])]
CITY_COUNTRY_lat_lon = CITY_COUNTRY_lat_lon.drop(columns=['Latitude', 'Longitude'])

In [68]:
#save the updates to the city info table
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','CITY_ADD_2022_09'))
CITY_COUNTRY_lat_lon.to_excel('CITY_COUNTRY_lat_lon.xlsx', index = False)

In [41]:
for count, CITY_COUNTRY in enumerate(new_cities_grid_refs.CITY_COUNTRY):
    grid_ref = new_cities_grid_refs[new_cities_grid_refs['CITY_COUNTRY'] ==  CITY_COUNTRY].Grid_Ref.values[0]
    CITY_COUNTRY_lat_lon.loc[CITY_COUNTRY_lat_lon['CITY_COUNTRY'] == CITY_COUNTRY, 'Grid_Ref'] = grid_ref
    
CITY_COUNTRY_lat_lon.loc[CITY_COUNTRY_lat_lon['CITY_COUNTRY'] == 'BLOEMFONTEIN_SOUTH_AFRICA', 'Grid_Ref'] = 'h20v11'

In [44]:
#read in the extention table 
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','CITY_ADD_2022_09'))
grid_refs2 = pd.read_excel('grid_refs.xlsx', index_col=None)

In [54]:
new_grid_list = []
for count, gf in enumerate(set(grid_refs2.Grid_Ref)):
    if gf not in CITY_COUNTRY_lat_lon.Grid_Ref:
        new_grid_list.append(gf)


In [74]:
# aqua only 
# all parameters, all cities
all_parameters_all_cities_df = pd.DataFrame()

for i in range(len(CITY_COUNTRY_lat_lon)):
    CITY_COUNTRY = CITY_COUNTRY_lat_lon.CITY_COUNTRY[i]
    city_name = CITY_COUNTRY_lat_lon.City[i]
    #koppen_2_letter = CITY_COUNTRY_lat_lon.Koppen_Geiger_2_Letter[i]

    #read in the SUHI parameters  
    os.chdir(os.path.join('E:\\','City_Data','MODIS_8_day_LST', CITY_COUNTRY, 'MYD11A2'))    
    pickle_name = 'Parameters_Day_{}_{}.pkl'.format(CITY_COUNTRY, 'MYD11A2')
    with open(pickle_name, 'rb') as f:
        aqua_parameters_day = pickle.load(f)
    pickle_name = 'Parameters_Night_{}_{}.pkl'.format(CITY_COUNTRY, 'MYD11A2')
    with open(pickle_name, 'rb') as f:
        aqua_parameters_night = pickle.load(f)
    
    aqua_parameters_day['Overpass'] = '13:30'    
    aqua_parameters_day['City'] = city_name
    aqua_parameters_day['CITY_COUNTRY'] = CITY_COUNTRY
    
    aqua_parameters_night['Overpass'] = '01:30'    
    aqua_parameters_night['City'] = city_name
    aqua_parameters_night['CITY_COUNTRY'] = CITY_COUNTRY
    #renames a0s to a0 so can be joined into same df
    aqua_parameters_day = aqua_parameters_day.rename(columns={"day_a0": "a0"})   
    aqua_parameters_night = aqua_parameters_night.rename(columns={"night_a0": "a0"}) 

    all_parameters_all_cities_df = pd.concat([all_parameters_all_cities_df, aqua_parameters_day])
    all_parameters_all_cities_df = pd.concat([all_parameters_all_cities_df, aqua_parameters_night])

##save the table as pickle
os.chdir(os.path.join(et.io.HOME, 'Documents', 'Python_Scripts', 'PROJECT','UHI_Project_Pickle_Files','All_cities'))   
with open('all_parameters_all_cities_df.pkl', 'wb') as f:
    pickle.dump(all_parameters_all_cities_df, f)

In [75]:
all_parameters_all_cities_df

,Filename,Date,Year,Month,a0,day_x0,day_y0,day_phi,day_ax,day_ay,...,night_y0,night_phi,night_ax,night_ay,night_std_dev_a0,night_std_dev_x0,night_std_dev_y0,night_std_dev_phi,night_std_dev_ax,night_std_dev_ay
0,MYD11A2.A2020345.h19v10.006.2020357230742.hdf,10/12/2020,2020,12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,MYD11A2.A2020353.h19v10.006.2020365040234.hdf,18/12/2020,2020,12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,MYD11A2.A2020337.h19v10.006.2020347085225.hdf,02/12/2020,2020,12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,MYD11A2.A2020329.h19v10.006.2020338110214.hdf,24/11/2020,2020,11,5.201033,4.8,2.795552,7.526327,19.088007,3.176753,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,MYD11A2.A2020321.h19v10.006.2020338021938.hdf,16/11/2020,2020,11,6.247544,4.8,-1.868706,160.059307,9.169199,4.731125,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
772,MYD11A2.A2020329.h11v05.006.2020338105740.hdf,24/11/2020,2020,11,1.916015,NaN,NaN,NaN,NaN,NaN,...,-0.486849,60.783368,6.612335,7.174910,0.211247,0.388935,0.371921,54.844186,0.729598,0.793047
773,MYD11A2.A2020337.h11v05.006.2020347085348.hdf,02/12/2020,2020,12,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.091097,0.698414,1.277029,12.239967,1.400639,2.553833
774,MYD11A2.A2020345.h11v05.006.2020357225036.hdf,10/12/2020,2020,12,1.699375,NaN,NaN,NaN,NaN,NaN,...,0.713662,176.229925,4.831825,7.562064,0.148822,0.212017,0.330927,7.666367,0.422798,0.666427
775,MYD11A2.A2020353.h11v05.006.2020365041111.hdf,18/12/2020,2020,12,1.446040,NaN,NaN,NaN,NaN,NaN,...,0.835656,-40.622106,23.260291,5.985690,0.147685,0.924498,0.808261,2.299315,2.448135,0.612189


In [73]:
aqua_parameters_day

,Filename,Date,Year,Month,a0,day_x0,day_y0,day_phi,day_ax,day_ay,...,method_2_SUHI,buffer_5km_SUHI,quantile_a0,quantile_75,city_pixels_used,rural_corr,eccentricity,Overpass,City,CITY_COUNTRY
0,MYD11A2.A2002185.h11v05.006.2015146152604.hdf,04/07/2002,2002,07,4.879826,-4.041002,-2.287751,124.057068,4.972913,17.455582,...,3.849333,2.861526,6.406,3.71,335.0,0.485714,NaN,13:30,Winston-Salem,WINSTON_SALEM_USA
1,MYD11A2.A2002193.h11v05.006.2015146152905.hdf,12/07/2002,2002,07,5.517651,-1.270763,-3.033778,72.544782,17.908622,9.392940,...,5.414051,3.816286,7.316,4.78,337.0,0.541225,NaN,13:30,Winston-Salem,WINSTON_SALEM_USA
2,MYD11A2.A2002201.h11v05.006.2015146153243.hdf,20/07/2002,2002,07,5.290055,2.280256,-1.750291,-77.588074,12.269695,6.607279,...,3.377846,2.738095,5.780,3.75,291.0,0.303580,0.640946,13:30,Winston-Salem,WINSTON_SALEM_USA
3,MYD11A2.A2002209.h11v05.006.2015152152207.hdf,28/07/2002,2002,07,8.097720,0.427473,-0.478637,-171.069976,6.418994,13.533167,...,5.703925,4.209611,8.954,5.96,267.0,0.445296,NaN,13:30,Winston-Salem,WINSTON_SALEM_USA
4,MYD11A2.A2002217.h11v05.006.2015149013834.hdf,05/08/2002,2002,08,6.068176,0.182059,-1.776727,-9.403910,8.247973,12.916733,...,4.860730,3.535876,7.076,4.06,337.0,0.539715,NaN,13:30,Winston-Salem,WINSTON_SALEM_USA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
813,MYD11A2.A2020329.h11v05.006.2020338105740.hdf,24/11/2020,2020,11,NaN,NaN,NaN,NaN,NaN,NaN,...,1.804517,1.149369,2.204,1.14,337.0,0.316549,NaN,13:30,Winston-Salem,WINSTON_SALEM_USA
814,MYD11A2.A2020337.h11v05.006.2020347085348.hdf,02/12/2020,2020,12,1.744301,0.256200,-0.070669,142.575245,6.042679,10.091155,...,1.051227,0.897075,1.864,1.04,337.0,0.197744,0.961205,13:30,Winston-Salem,WINSTON_SALEM_USA
815,MYD11A2.A2020345.h11v05.006.2020357225036.hdf,10/12/2020,2020,12,1.391179,0.805333,1.247560,139.385211,8.685991,20.990936,...,1.325649,1.106441,2.392,1.12,337.0,0.215634,0.763738,13:30,Winston-Salem,WINSTON_SALEM_USA
816,MYD11A2.A2020353.h11v05.006.2020365041111.hdf,18/12/2020,2020,12,2.194108,-2.075504,-2.610644,-156.358800,13.688655,10.632186,...,1.639206,1.137796,2.428,1.50,337.0,0.569787,NaN,13:30,Winston-Salem,WINSTON_SALEM_USA


In [76]:
all_parameters_all_cities_df = pd.concat([all_parameters_all_cities_df, aqua_parameters_day])

In [78]:
all_parameters_all_cities_df.CITY_COUNTRY

0          HUAMBO_ANGOLA
1          HUAMBO_ANGOLA
2          HUAMBO_ANGOLA
3          HUAMBO_ANGOLA
4          HUAMBO_ANGOLA
             ...        
813    WINSTON_SALEM_USA
814    WINSTON_SALEM_USA
815    WINSTON_SALEM_USA
816    WINSTON_SALEM_USA
817    WINSTON_SALEM_USA
Name: CITY_COUNTRY, Length: 136735, dtype: object